In [1]:
import torch
import numpy as np

## 1. indexing and slicing

In [2]:
tensor = torch.rand(2, 3)
print(f"1st row: {tensor[0]}")
print(f"1st col: {tensor[:, 0]}")
print(f"last col: {tensor[:, -1]}")
tensor[:, 1] = 0
print(tensor, tensor.device)

1st row: tensor([0.3466, 0.8739, 0.2632])
1st col: tensor([0.3466, 0.3762])
last col: tensor([0.2632, 0.8377])
tensor([[0.3466, 0.0000, 0.2632],
        [0.3762, 0.0000, 0.8377]]) cpu


## 2. concatenation and repeating
- <font color=blue>**torch.cat(tensor sequences, dim=0)**</font>: <font color=green>dim的取值范围是[0, number of dims-1]</font>。在指定维度上堆叠tensor sequence。output的dim数量不变，指定dim的length变大。
  - tensor的dim数>=2时，<font color=blue>**hstack(a, b)**</font>相当于cat((a, b), dim=1)
  - tensor的dim数>=2时，<font color=blue>**vstack(a, b)**</font>相当于cat((a, b), dim=0)
- <font color=blue>**torch.stack(tensor sequences, dim=0)**</font>: <font color=green>dim的取值范围是[0, number of dims]</font>。新增1维，在新增的维度上堆叠指定的tensor sequence。dim数量+1.<font color=green>相当于在指定维度上对所有tensor sequence做unsqueeze，然后将他们做torch.cat</font>
- <font color=blue>**tensor.repeat()**</font>: 复制指定维度，维度的数量可以扩张。类似numpy.tile
  - 将选定维度上的data整体repeat，[1, 2, 3]repeat 2次就是[1, 2, 3, 1, 2, 3]
  - 如果参数的维度数量>tensor维度数量，就在dim0位置插入新的dim，然后repeat
- <font color=blue>**tensor.repeat_interleave()**</font>: 重复每个维度上的元素，类似numpy.repeat
  - 将选定维度上的元素逐个repeat，[1, 2, 3]repeat 2次就是[1, 1, 2, 2, 3, 3]

### 2.1 torch.cat将tensor sequence在已有维度上做堆叠
- 除了参数指定的维度，其他维度上的length必须一样，或者是empty tensor
- dim参数取值不能超过input的维度数

In [3]:
x = torch.rand(2, 2)
x, x.shape

(tensor([[0.2625, 0.2303],
         [0.7754, 0.0500]]),
 torch.Size([2, 2]))

In [4]:
t1 = torch.cat([x, x], dim=1) # 在dim 1上堆，shape的dim 0大小不变
t2 = torch.cat([x, x], dim=0) # 在dim 0上堆，shape的dim 1大小不变
t1, t2, len(x.shape) == len(t1.shape) == len(t2.shape)

(tensor([[0.2625, 0.2303, 0.2625, 0.2303],
         [0.7754, 0.0500, 0.7754, 0.0500]]),
 tensor([[0.2625, 0.2303],
         [0.7754, 0.0500],
         [0.2625, 0.2303],
         [0.7754, 0.0500]]),
 True)

In [5]:
a = torch.ones(2, 3, 5)
b = torch.arange(30).view(2, 3, 5)
v = torch.vstack((a, b)) # concatenate dim0
v.shape, torch.all(v == torch.cat((a, b), dim=0)).item()

(torch.Size([4, 3, 5]), True)

In [6]:
h = torch.hstack((a, b)) # concatenate dim1
h.shape, torch.all(h == torch.cat((a, b), dim=1)).item()

(torch.Size([2, 6, 5]), True)

In [7]:
x = torch.arange(4).view(1, 2, 2)
a = torch.cat((x, x, x), 0)

## 和expand的区别：expand不copy，只是给多个重复的view
e = x.expand([2, 2, 2])

a, e, a.storage(), e.storage()

/tmp/ipykernel_18781/422504737.py:7: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a, e, a.storage(), e.storage()


(tensor([[[0, 1],
          [2, 3]],
 
         [[0, 1],
          [2, 3]],
 
         [[0, 1],
          [2, 3]]]),
 tensor([[[0, 1],
          [2, 3]],
 
         [[0, 1],
          [2, 3]]]),
  0
  1
  2
  3
  0
  1
  2
  3
  0
  1
  2
  3
 [torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 12],
  0
  1
  2
  3
 [torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 4])

In [8]:
b = torch.cat((x, x, x), 1)
b, a.data_ptr() == b.data_ptr()

(tensor([[[0, 1],
          [2, 3],
          [0, 1],
          [2, 3],
          [0, 1],
          [2, 3]]]),
 False)

### 2.2 torch.stack将tensor sequence堆叠到新的维度
- 相当于在指定维度上对所有tensor sequence做unsqueeze，然后做torch.cat
- 被stack的tensors要有相同的shape
- output维度+1

In [9]:
y = torch.zeros(2, 3) 
z = torch.arange(1, 7).reshape(2, 3)
y, z

(tensor([[0., 0., 0.],
         [0., 0., 0.]]),
 tensor([[1, 2, 3],
         [4, 5, 6]]))

In [10]:
t3 = torch.stack([y, z], dim=0)
t4 = torch.stack([y, z], dim=1)
t5 = torch.stack([y, z], dim=2)  # dim取值范围[-3, 2]
print(t3, t3.shape)
print(t4, t4.shape)
print(t5, t5.shape)
print(f"# dim of t3,t4,t5: {len(t3.shape), len(t4.shape), len(t5.shape)}")

tensor([[[0., 0., 0.],
         [0., 0., 0.]],

        [[1., 2., 3.],
         [4., 5., 6.]]]) torch.Size([2, 2, 3])
tensor([[[0., 0., 0.],
         [1., 2., 3.]],

        [[0., 0., 0.],
         [4., 5., 6.]]]) torch.Size([2, 2, 3])
tensor([[[0., 1.],
         [0., 2.],
         [0., 3.]],

        [[0., 4.],
         [0., 5.],
         [0., 6.]]]) torch.Size([2, 3, 2])
# dim of t3,t4,t5: (3, 3, 3)


In [11]:
x = torch.tensor([[[1, 2, 0], 
                   [5, 6, 7]],
                  
                  [[8, 9, 0], 
                   [3, 4, 5]]])

y = torch.tensor([[[1, 1, 1], 
                   [2, 2, 2]],
                  
                  [[0, 0, 0], 
                   [9, 9, 9]]])
x.shape, y.shape

(torch.Size([2, 2, 3]), torch.Size([2, 2, 3]))

In [12]:
t1 = torch.stack([x, y], dim=0) # 在dim 0上堆，shape的dim 1,2大小不变
t2 = torch.stack([x, y], dim=1) # 在dim 1上堆，shape的dim 0,2大小不变
t3 = torch.stack([x, y], dim=2) # 在dim 2上堆，shape的dim 1,2大小不变
print('在dim 0上堆:\n', t1, t1.shape)
print('在dim 1上堆:\n', t2, t2.shape)
print('在dim 2上堆:\n', t3, t3.shape)

在dim 0上堆:
 tensor([[[[1, 2, 0],
          [5, 6, 7]],

         [[8, 9, 0],
          [3, 4, 5]]],


        [[[1, 1, 1],
          [2, 2, 2]],

         [[0, 0, 0],
          [9, 9, 9]]]]) torch.Size([2, 2, 2, 3])
在dim 1上堆:
 tensor([[[[1, 2, 0],
          [5, 6, 7]],

         [[1, 1, 1],
          [2, 2, 2]]],


        [[[8, 9, 0],
          [3, 4, 5]],

         [[0, 0, 0],
          [9, 9, 9]]]]) torch.Size([2, 2, 2, 3])
在dim 2上堆:
 tensor([[[[1, 2, 0],
          [1, 1, 1]],

         [[5, 6, 7],
          [2, 2, 2]]],


        [[[8, 9, 0],
          [0, 0, 0]],

         [[3, 4, 5],
          [9, 9, 9]]]]) torch.Size([2, 2, 2, 3])


### 2.3 tensor.repeat(): 重复堆叠tensor.类似numpy.tile
- <font color=red>repeat会copy data</font>
- 参数数量(number)决定在多少个维度上做堆叠，参数个数可以大于tensor维数\
- 每个参数值(value)指定对应维度的堆叠次数\
- 如tensor有d维，后d个参数指定d维上堆叠的次数，其他参数决定前序扩张维度上的堆叠次数

In [13]:
x = torch.tensor([1, 2, 3])  # dim=1, shape=(n,)
y0 = x.repeat(3)
y0, y0.shape

(tensor([1, 2, 3, 1, 2, 3, 1, 2, 3]), torch.Size([9]))

In [14]:
y1 = x.repeat(3, 2)          # dim=2, shape=(3, 2*n)
y1, y1.shape

(tensor([[1, 2, 3, 1, 2, 3],
         [1, 2, 3, 1, 2, 3],
         [1, 2, 3, 1, 2, 3]]),
 torch.Size([3, 6]))

In [15]:
y2 = x.repeat(2, 1, 2)       # dim=3, shape=(2, 1, 2*n)
y2, y2.shape

(tensor([[[1, 2, 3, 1, 2, 3]],
 
         [[1, 2, 3, 1, 2, 3]]]),
 torch.Size([2, 1, 6]))

In [16]:
x = torch.tensor([[1, 2, 0], 
                  [5, 6, 7]])  # dim=d=2, shape=(m, n)=(2, 3)
y1 = x.repeat(3, 2)                       # dim=2, shape=(3*m, 2*n)
y2 = x.repeat(2, 2, 3)                    # dim=3, shape=(2, 2*m, 3*n)
y3 = x.repeat(2, 2, 1, 1)                 # dim=4, shape=(2, 2, 1*m, 1*n)
print(y1, y1.shape)
print(y2, y2.shape)
print(y3, y3.shape)

tensor([[1, 2, 0, 1, 2, 0],
        [5, 6, 7, 5, 6, 7],
        [1, 2, 0, 1, 2, 0],
        [5, 6, 7, 5, 6, 7],
        [1, 2, 0, 1, 2, 0],
        [5, 6, 7, 5, 6, 7]]) torch.Size([6, 6])
tensor([[[1, 2, 0, 1, 2, 0, 1, 2, 0],
         [5, 6, 7, 5, 6, 7, 5, 6, 7],
         [1, 2, 0, 1, 2, 0, 1, 2, 0],
         [5, 6, 7, 5, 6, 7, 5, 6, 7]],

        [[1, 2, 0, 1, 2, 0, 1, 2, 0],
         [5, 6, 7, 5, 6, 7, 5, 6, 7],
         [1, 2, 0, 1, 2, 0, 1, 2, 0],
         [5, 6, 7, 5, 6, 7, 5, 6, 7]]]) torch.Size([2, 4, 9])
tensor([[[[1, 2, 0],
          [5, 6, 7]],

         [[1, 2, 0],
          [5, 6, 7]]],


        [[[1, 2, 0],
          [5, 6, 7]],

         [[1, 2, 0],
          [5, 6, 7]]]]) torch.Size([2, 2, 2, 3])


### 2.4 tensor.repeat_interleave(\<tensor>, \<n>, dim=?)
- 不指定参数dim=？时，输出会被flattened成1维\
- 参数dim决定元素在哪个维度上重复，n决定重复次数
- 第2个参数指定repeat的次数，如果是int，就是每个元素重复次数；如果是tensor，那么tensor的长度必须等于input tensor的长度，每个tensor的值决定input tensor中每个元素重复多少次

In [17]:
x = torch.tensor([[1, 2], [3, 4], [5, 6]])

y1 = x.repeat_interleave(2)
print('flattened, 重复2次:',y1)

flattened, 重复2次: tensor([1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6])


In [18]:
y3 = x.repeat_interleave(2, dim=0)
print('沿dim0，重复2次:','\n', y3)
y4 = x.repeat_interleave(3, dim=1)
print('沿dim1，重复3次:','\n', y4)

沿dim0，重复2次: 
 tensor([[1, 2],
        [1, 2],
        [3, 4],
        [3, 4],
        [5, 6],
        [5, 6]])
沿dim1，重复3次: 
 tensor([[1, 1, 1, 2, 2, 2],
        [3, 3, 3, 4, 4, 4],
        [5, 5, 5, 6, 6, 6]])


In [19]:
# 沿dim0，分别重复1， 2， 1次
y5 = x.repeat_interleave(torch.tensor([1, 2, 1]), dim=0)
y5

tensor([[1, 2],
        [3, 4],
        [3, 4],
        [5, 6]])